In [1]:
import os.path
import sys
sys.path.insert(0, '../../')
from questions_construction.questions import FLUENT_TYPES_LIST
from questions_construction.main import PLAN_LENGTHS, QUESTION_CATEGORIES
from questions_construction.domains import DOMAIN_NAMES
from analysis.model_performances import * #gather_questions, TRANSPORTATION_DOMAINS, NON_TRANSPORTATION_DOMAINS
from common import *
import random
from collections import defaultdict
from copy import deepcopy
import itertools
from analysis.model_performances import *
import pandas as pd
from helpers import *

In [12]:
questions_dir = f'{DATA_PATH}/questions_m1'
name = 'dataset_ids.test.pruned'
test_ids = open_jsonl(f'{DATA_PATH}/{name}.jsonl')
train_ids = open_jsonl(f'{DATA_PATH}/dataset_ids.train.jsonl')
questions_by_id = gather_questions(questions_dir)

save_main_dir = f'{STATISTICS_PATH}.{name}'
os.makedirs(os.path.join(save_main_dir, 'tables'), exist_ok=True)

questions gathered


In [13]:
questions_by_id = {k: v[WITHOUT_RANDOM_SUB] for k, v in questions_by_id.items()}

In [14]:
for k, d in questions_by_id.items():
    if d['id'] in test_ids:
        d['split_type'] = 'test'
    else:
        d['split_type'] = 'train'

In [24]:
by_category = defaultdict(int)
save_key = 'dataset_split'
for d in questions_by_id.values():
    by_category[(d['question_category'], d['split_type'])]+=1

df_by_category = pd.DataFrame(index=sorted(QUESTION_CATEGORIES), columns=['test', 'train'])
for k, v in by_category.items():
    df_by_category.loc[k[0], k[1]] = v
df_by_category

/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:44: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)


In [25]:
latex_table = to_latex_table(df_by_category, 'dataset split')
with open(os.path.join(save_main_dir, 'tables', f'{save_key}.tex'), 'w') as f:
    f.write(latex_table)

/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:44: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)


In [26]:
by_category = defaultdict(int)
save_key = 'dataset_split_by_answer_type'
for d in questions_by_id.values():
    by_category[(d['question_category'], d['split_type'], d['answer_type'])]+=1

df_by_category = pd.DataFrame(index=sorted(QUESTION_CATEGORIES), columns=['test-true_false_answer', 'test-free_answer', 'train-true_false_answer', 'train-free_answer'])
for k, v in by_category.items():
    df_by_category.loc[k[0], k[1]+'-'+k[2]] = v
df_by_category

,test-true_false_answer,test-free_answer,train-true_false_answer,train-free_answer
action_executability,240,119,2360,2481
effects,957,108,8243,1842
fluent_tracking,1600,130,15801,5870
hallucination,240,130,7660,13426
numerical_reasoning,240,120,12760,6380
object_tracking,1598,120,8696,1180
state_tracking,153,101,1147,1849


In [27]:
latex_table = to_latex_table(df_by_category, 'dataset split')

with open(os.path.join(save_main_dir, 'tables', f'{save_key}.tex'), 'w') as f:
    f.write(latex_table)

/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:44: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)


In [20]:
by_category_by_fluents = defaultdict(int)
for d in questions_by_id.values():
    if d['fluent_type'] not in ('None', None):
        by_category_by_fluents[(d['question_category'], d['fluent_type'], d['split_type'])]+=1
        
columns = []
for c in ['base_fluents', 'persistent_fluents', 'derived_fluents', 'static_fluents']:
    columns.append(c+'-test')
    columns.append(c+'-train')
    
index = ['object_tracking', 'fluent_tracking', 'effects']
df = pd.DataFrame(index=index, columns=columns)
rows = []
for k, v in by_category_by_fluents.items():
    df.loc[k[0], k[1]+'-'+k[2]] = v
df

,base_fluents-test,base_fluents-train,persistent_fluents-test,persistent_fluents-train,derived_fluents-test,derived_fluents-train,static_fluents-test,static_fluents-train
object_tracking,382,1388,480,3125,264,1655,472,2528
fluent_tracking,427,3698,442,7358,431,5045,430,5570
effects,239,1761,240,2360,240,2060,238,2062


In [21]:
latex_table = to_latex_table(df, 'dataset split by fluents').replace('_', ' ')
save_key = 'fluent_split'
with open(os.path.join(save_main_dir, 'tables', f'{save_key}.tex'), 'w') as f:
    f.write(latex_table)

/Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:44: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)


# Composite Questions

In [3]:
questions_dir = f'{DATA_PATH}/questions.composite.test_train'
data_composite_train = gather_questions(questions_dir)

missing /Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/data/questions.composite.test_train_composite/without_random_sub/blocksworld/Instance_1.jsonl
missing /Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/data/questions.composite.test_train_composite/without_random_sub/blocksworld/Instance_2.jsonl
missing /Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/data/questions.composite.test_train_composite/without_random_sub/blocksworld/Instance_3.jsonl
missing /Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/data/questions.composite.test_train_composite/without_random_sub/blocksworld/Instance_4.jsonl
missing /Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/data/questions.composite.test_train_composite/without_random_sub/blocksworld/Instance_5.jsonl
missing /Users/paveldolin/dev/research/current/reasoning_about_actions/pipeline/data/questions.composite.test_train_composite/with

In [4]:
len(data_composite_train)

15199

In [5]:
359+1065+1730+370+360+1718+254+360

6216

In [6]:
4841 + 10085 + 21671 + 21086 + 19140 + 9876 + 2996 + 15199

104894